In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [2]:
!gdown 1GSAF_u7Gxz0Wcxjd-7zL1Gd2WfC0s7Zz
!unzip wind-turbine-damage-challenges.zip
!rm wind-turbine-damage-challenges.zip

Downloading...
From (original): https://drive.google.com/uc?id=1GSAF_u7Gxz0Wcxjd-7zL1Gd2WfC0s7Zz
From (redirected): https://drive.google.com/uc?id=1GSAF_u7Gxz0Wcxjd-7zL1Gd2WfC0s7Zz&confirm=t&uuid=902b8aab-97e8-4fc8-a22f-b0ff39474975
To: /kaggle/working/wind-turbine-damage-challenges.zip
100%|██████████████████████████████████████| 1.58G/1.58G [00:16<00:00, 93.4MB/s]
Archive:  wind-turbine-damage-challenges.zip
  inflating: Wind Turbine damage/README.roboflow.txt  
  inflating: Wind Turbine damage/test/20240120_150729_jpg.rf.5925f54483de8cc52f75fdd1dc0ec3a0.jpg  
  inflating: Wind Turbine damage/test/20240120_153628_jpg.rf.67030e3c4592587d26d7762e8e091020.jpg  
  inflating: Wind Turbine damage/test/20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397d4ad42f2e.jpg  
  inflating: Wind Turbine damage/test/20240120_153711_jpg.rf.3a50aa9daa881abe4296c7f12b03fcf9.jpg  
  inflating: Wind Turbine damage/test/20240120_153748-0-_jpg.rf.6c50a8089bf3ccd32bb61e347e9f4019.jpg  
  inflating: Wind Turbine da

In [3]:
import os
import json
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import shutil
from ultralytics import YOLO
import pandas as pd

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
def load_coco_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        return json.load(f)

def convert_coco_to_yolo(coco_bbox, img_width, img_height):
    x, y, w, h = coco_bbox
    x_center = x + w/2
    y_center = y + h/2

    # Нормализация координат для YOLO формата
    x_center = x_center / img_width
    y_center = y_center / img_height
    w = w / img_width
    h = h / img_height

    return [x_center, y_center, w, h]

def process_image(image_path, annotation_data, type_folder, output_size=640, overlap=0.5):
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    image_id = next(img_info['id'] for img_info in annotation_data['images']
                   if img_info['file_name'] == os.path.basename(image_path))
    annotations = [ann for ann in annotation_data['annotations'] if ann['image_id'] == image_id]

    output_img_dir = Path(f'processed_dataset/images/{type_folder}')
    output_label_dir = Path(f'processed_dataset/labels/{type_folder}')
    output_img_dir.mkdir(parents=True, exist_ok=True)
    output_label_dir.mkdir(parents=True, exist_ok=True)

    step = int(output_size * (1 - overlap))

    counter = 0

    # Скользящее окно
    for y in range(0, img_height - output_size + 1, step):
        for x in range(0, img_width - output_size + 1, step):
            crop = img[y:y+output_size, x:x+output_size]

            yolo_annotations = []

            for ann in annotations:
                bbox = ann['bbox']
                category_id = ann['category_id']-1

                # Проверяем, находится ли bbox в текущем окне
                if (x <= bbox[0] <= x + output_size and
                    y <= bbox[1] <= y + output_size):

                    # Корректируем координаты bbox относительно окна
                    new_bbox = [
                        bbox[0] - x,
                        bbox[1] - y,
                        min(bbox[2], output_size - (bbox[0] - x)),  # Обрезаем если выходит за границы
                        min(bbox[3], output_size - (bbox[1] - y))   # Обрезаем если выходит за границы
                    ]

                    yolo_bbox = convert_coco_to_yolo(new_bbox, output_size, output_size)

                    # Проверяем, что все координаты в допустимом диапазоне [0, 1]
                    if all(0 <= coord <= 1 for coord in yolo_bbox):
                        yolo_annotations.append(f"{category_id} {' '.join(map(str, yolo_bbox))}")

            if yolo_annotations:
                output_img_path = output_img_dir / f"{Path(image_path).stem}_{counter}.jpg"
                cv2.imwrite(str(output_img_path), crop)

                output_label_path = output_label_dir / f"{Path(image_path).stem}_{counter}.txt"
                with open(output_label_path, 'w') as f:
                    f.write('\n'.join(yolo_annotations))

                counter += 1

def process_dataset(input_dir, annotation_path, type_folder):
    # Загружаем аннотации COCO
    annotation_data = load_coco_annotations(annotation_path)

    for img_path in tqdm(list(Path(input_dir).glob('*.jpg'))):
        process_image(str(img_path), annotation_data, type_folder)


def train_yolov8():
    # Создаем файл конфигурации датасета
    dataset_yaml = """
    path: /kaggle/working/processed_dataset/
    train: images/train
    val: images/valid
    names:
      0: corrosion
      1: lightning
      2: lightning receptor
      3: missing teeth
      4: patch
    """

    with open('/kaggle/working/dataset.yaml', 'w') as f:
        f.write(dataset_yaml)

    # MARAT: model = YOLO(yolov8l.pt) model.train(data='dataset.yaml', epochs=200, imgsz=640, batch=8, save_period=25, exist_ok=True, project="detect_turbines", name="yolo8-detect")
    # ЧТОБЫ ПРОДОЛЖИТЬ НАДО В КОННЦЕ model.train(..., resume=True) ДОБАВИТЬ resume=True
    # ГОША: тебе нужно закинуть epoch60.pt в корень и запустить
    model = YOLO('/kaggle/input/epoch90/pytorch/default/1/epoch90.pt') #  yolov8n.pt

    model.train(data='dataset.yaml', epochs=150, imgsz=640, batch=16, save_period=15, exist_ok=True,project="detect_turbines", name="yolo8-detect", resume = True)

In [5]:
# Один раз запускается (второй раз не перезапускать!!!)
process_dataset("Wind Turbine damage/train", "Wind Turbine damage/train/_annotations.coco.json", "train")
process_dataset("Wind Turbine damage/valid", "Wind Turbine damage/valid/_annotations.coco.json", "valid")

100%|██████████| 271/271 [00:16<00:00, 16.86it/s]


In [9]:
train_yolov8()

Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/epoch90/pytorch/default/1/epoch90.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8-detect, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, 

train: Scanning /kaggle/working/processed_dataset/labels/train.cache... 9250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9250/9250 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 573.2±278.1 MB/s, size: 78.8 KB)


val: Scanning /kaggle/working/processed_dataset/labels/valid.cache... 2212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2212/2212 [00:00<?, ?it/s]


Plotting labels to detect_turbines/yolo8-detect/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to detect_turbines/yolo8-detect
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150      2.08G      1.329     0.8197     0.9738         45        640:  10%|▉         | 56/579 [00:09<01:32,  5.67it/s]


KeyboardInterrupt: 

## Submit

In [7]:
!gdown 1ePuyQSNw6FrvfsOcL0h2Z79d-VD2bgwf
!unzip sample_submission_with_errors.csv.zip
!rm sample_submission_with_errors.csv.zip

Downloading...
From: https://drive.google.com/uc?id=1ePuyQSNw6FrvfsOcL0h2Z79d-VD2bgwf
To: /kaggle/working/sample_submission_with_errors.csv.zip
100%|██████████████████████████████████████| 55.4k/55.4k [00:00<00:00, 64.1MB/s]
Archive:  sample_submission_with_errors.csv.zip
  inflating: sample_submission_with_errors.csv  
  inflating: __MACOSX/._sample_submission_with_errors.csv  


In [8]:
def process_image_for_inference(image_path, model, output_size=640, overlap=0.5):
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    # Создаем временную директорию для обработанных изображений
    temp_dir = Path('temp_inference')
    temp_dir.mkdir(exist_ok=True)

    # Размер шага с учетом перекрытия
    step = int(output_size * (1 - overlap))

    class_names = [
    "corrosion",
    "lightning",
    "lightning receptor",
    "missing teeth",
    "patch"
    ]

    predictions = []

    # Скользящее окно
    for y in range(0, img_height - output_size + 1, step):
        for x in range(0, img_width - output_size + 1, step):
            crop = img[y:y+output_size, x:x+output_size]

            temp_img_path = temp_dir / f"{Path(image_path).stem}_temp.jpg"
            cv2.imwrite(str(temp_img_path), crop)

            results = model(temp_img_path, verbose=False)

            for result in results:
                boxes = result.boxes

                for box in boxes:
                    class_index = int(box.cls[0].cpu().numpy())
                    class_name = class_names[class_index]
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = float(box.conf[0].cpu().numpy())

                    # Переводим координаты в исходную систему координат
                    x1 = float(x1 + x)
                    y1 = float(y1 + y)
                    x2 = float(x2 + x)
                    y2 = float(y2 + y)

                    # Формируем bbox в формате [x, y, width, height]
                    bbox = [x1, y1, x2 - x1, y2 - y1]

                    predictions.append([f'"{class_name}"'] + bbox + [conf])

    shutil.rmtree(temp_dir)

    return predictions

def create_submission_file(predictions, output_file='submission.csv'):
    df = pd.DataFrame(predictions)
    df.to_csv(output_file, index=False, sep=',')

In [12]:
model = YOLO('/kaggle/working/detect_turbines/yolo8-detect/weights/last.pt')

df_example = pd.read_csv("/kaggle/input/sample/sample_submission_with_errors.csv")
images = df_example["image_id"].to_list()

results = []
for image_id in tqdm(images):
    img_path = os.path.join("Wind Turbine damage", "test", image_id)
    predictions = process_image_for_inference(str(img_path), model)
    results.append({"image_id": str(image_id), "objects": str(predictions)})

create_submission_file(results)

100%|██████████| 151/151 [02:31<00:00,  1.01s/it]


In [ ]:
df_sub = pd.read_csv("/content/submission.csv") # НУЖНО СКАЧАТЬ submission.csv и удалить через ctrl F все кавычки одинарные иначе сабмит не зайдет '

In [ ]:
df_example.head(5)

In [ ]:
df_sub.head(5)